# Statewide SHN and GTFS LOS data 
Let's make some optional mapping about where the SHN and trans service interact 

In [10]:
import geopandas as gpd
import ibis
import pandas as pd
import pydeck as pdk
from ipyleaflet import Map, GeoData, basemaps, LayersControl
import json

In [11]:
web_mercator = 'epsg:4326'
ca_albers = 'epsg:3488'

In [12]:
shn = gpd.read_file('https://opendata.arcgis.com/datasets/77f2d7ba94e040a78bfbe36feb6279da_0.geojson')

In [13]:
conn = ibis.bigquery.connect(
    project_id='cal-itp-data-infra',
    dataset_id='gtfs_schedule')

In [14]:
stops = conn.table('stops')

In [15]:
stops_df = stops.execute(limit=None)

In [16]:
stops_gdf = gpd.GeoDataFrame(stops_df, geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat))

# Map of Stops and the SHN

In [25]:
shn_data = GeoData(geo_dataframe = shn,
                   style={'color': 'purple', 'opacity':3, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'State Highway Network')
stops_data = GeoData(geo_dataframe = stops_gdf.set_crs(web_mercator),
                        style={'color': 'black', 'radius':.1, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                        point_style={'radius': 1, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
                        name = 'Release')

In [26]:
m = Map(center=(36.7783,-118.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap)
m.add_layer(shn_data)
m.add_layer(stops_data)
m

Map(center=[36.7783, -118.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…